In [ ]:
import sys
sys.path.append("..")
from src.DataBaseManager import DBMS, QUERY_CATALOG
from config import LAND_COVER_PALETTE, LAND_COVER_LEGEND,LEGEND_TO_PALETTE
import contextily as ctx
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import pandas as pd
from tqdm import tqdm
from PIL import Image

from src.plotting.sankey import plot_sankey

import geemap

import ee
from src.utils import authenticate_Google_Earth_Engine as authenticate

from IPython.display import Image as IPy_Image




In [ ]:
authenticate()

In [ ]:
years = [str(year) for year in range(2016,2024)]



def brighten(band):
    alpha=0.13
    beta=0
    return np.clip(alpha*band+beta, 0,255)

def sql_list_strings(list_):
    return ','.join(["'"+str(i)+"'" for i in list_])

def get_LULC_for_chips(chipids,year,country):
    DB = DBMS()
    results = DB.read('GET_DW_LANDCOVER',{'_CHIPIDS_':sql_list_strings(chipids),
                                '_AREA_':country,
                                '_START_YEAR_':year,
                                '_END_YEAR_':year},
                                geom_query = True)   


    return results 


import numpy as np
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

def brighten(band, lower_percent=2, upper_percent=98):
    """
    Brighten the image by applying contrast stretching.
    """
    # Compute the lower and upper percentile
    lower = np.percentile(band, lower_percent)
    upper = np.percentile(band, upper_percent)
    
    # Clip the band to the lower and upper percentiles and scale it to 0-1
    band = np.clip((band - lower) / (upper - lower), 0, 1)
    
    return band

def enhance_image(vals):
    """
    Enhance the image by brightening and adjusting gamma.
    """
    bands = []
    for band in vals[:3]:
        band = brighten(band)
        bands.append(band)
    
    # Stack the bands back together
    enhanced_image = np.stack(bands, axis=0)
    
    # Apply gamma correction
    gamma = 1.2  # You can adjust this value to your preference
    enhanced_image = np.power(enhanced_image, gamma)
    
    return enhanced_image






# This is the cloud masking function provided by GEE but adapted for use in Python.
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask = mask.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)

def get_S2_image(results,year,data_dir):

    
    
    ROI = results.dissolve()
    fc = geemap.geopandas_to_ee(ROI[["geometries"]])
    #return fc
    # Define the geometry of the area for which you would like images.
    try:
        geom = ee.Geometry.Polygon(fc.geometry().getInfo()['coordinates'])
    except:
        geom = ee.Geometry.MultiPolygon(fc.geometry().getInfo()['coordinates'])
    # Call collection of satellite images.
    collection = (ee.ImageCollection("COPERNICUS/S2")
                # Select the Red, Green and Blue image bands, as well as the cloud masking layer.
                .select(['B4', 'B3', 'B2', 'QA60'])
                # Filter for images within a given date range.
                .filter(ee.Filter.date(f'{year}-01-01', f'{year}-12-31'))
                # Filter for images that overlap with the assigned geometry.
                .filterBounds(geom)
                # Filter for images that have less then 20% cloud coverage.
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                # Apply cloud mask.
                .map(maskS2clouds)
                )

    # Sort images in the collection by index (which is equivalent to sorting by date), 
    # with the oldest images at the front of the collection.
    # Convert collection into a single image mosaic where only images at the top of the collection are visible.
    image = collection.sort('system:index', opt_ascending=False).mosaic()


    scale = 50  # Scale in meters.
    region = geom.buffer(1000).bounds().getInfo()['coordinates']



    url = image.getDownloadURL({
        'scale': scale,
        'region': region,
        'format': 'GEO_TIFF',
        'bands': ['B4', 'B3', 'B2']
    })

    num = int(random.random()*1000)
    image_path = data_dir+f'sentinel2_image_{num}.tif'
    geemap.download_file(url, image_path)


    return image_path



def plot_sentinel_image(image_path, chipid,year,data_dir):

    # Open the downloaded image.
    with rasterio.open(image_path) as src:
        # Read the RGB bands
        vals = src.read([1, 2, 3])

        fig, ax = plt.subplots(figsize=(10, 10))
        plt.tight_layout()
        
        # Enhance the image
        enhanced_vals = enhance_image(vals)
        
        # Display the enhanced image
        chipid_str = data_dir.split('/')[-2]
        show(enhanced_vals, ax=ax, title=f"Sentinel-2 Image of {chipid_str} in {year}")

        # remove axis
        ax.axis('off')
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)

        # save image to png
        plt.savefig(data_dir + f'{year}_S2.png',bbox_inches='tight')
        plt.close(fig)

        return ax

    
def plot_LULC_map(gdf,chipid,year,data_dir='../plots/chips/'):

    colormap = {LAND_COVER_LEGEND[ix]: LAND_COVER_PALETTE[ix] for ix in LAND_COVER_PALETTE.keys()}
    # Ensure the 'name' column exists and maps to the correct land cover class
    gdf = gdf.sort_values(['data_origins','name'])
    gdf['color'] = gdf['name'].map(colormap)  # Default to black if not found
    fig, ax = plt.subplots(figsize=(10, 10))
    plt.tight_layout()
    
    # Plot the entire GeoDataFrame at once using the 'color' column for color coding
    ax2  = gdf.plot(ax=ax, color=gdf['color'], alpha=0.75)  # Adjust alpha as needed

    # Add the OSM Mapnik map as the base layer
    ctx.add_basemap(ax2, source=ctx.providers.OpenStreetMap.Mapnik, crs=gdf.crs.to_string())

    ax2.set_xlim(gdf.total_bounds[[0, 2]])
    ax2.set_ylim(gdf.total_bounds[[1, 3]])
    
    chipid_str = data_dir.split('/')[-3]
    
    plt.title(f"Land Use and Land Cover Map of {chipid_str} in {year}", fontweight='bold')

    # remove axis
    ax2.axis('off')

    ax2.spines['right'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)

    plt.tight_layout()
    # save image to png
    plt.savefig(data_dir + f'{year}_LULC.png',bbox_inches='tight')
    plt.close(fig)





def plot_year(chipid,year,country = "Denmark",intermediate_dir='./',gee_dir='./'):
    
    print("Getting LULC data")
    results = get_LULC_for_chips(chipids=chipid,year=year,country=country)

    print("Getting satellite image")
    image_path = get_S2_image(results,year,data_dir=gee_dir)

    ax = plot_LULC_map(results,chipid=chipid,year=year,data_dir=intermediate_dir)
    ax = plot_sentinel_image(image_path,chipid,year,data_dir=intermediate_dir)

def cm_to_inch(cm):
    return cm * 0.393701

def create_intermediate_plots(chipids,years,country = "Denmark",intermediate_dir='./',gee_dir='./'):
    for year in tqdm(years):
        year = str(year)
        plot_year(chipid=chipids,year=year,country=country,intermediate_dir=intermediate_dir,gee_dir=gee_dir)


def create_massive_subplot(chipid,years,data_dir='../plots/chips/',intermediate_dir='./'):
    plt.style.use('default')
    vert = 24/8*len(years)

    aspect_ratio = 9/vert
    fixed_width_cm = 13
    fixed_width_inch = cm_to_inch(fixed_width_cm)

    height_inch = fixed_width_inch / aspect_ratio


    fig, ax = plt.subplots(len(years),2, figsize=(fixed_width_inch, height_inch))

    for year in years:
        
        img = plt.imread(f'{intermediate_dir}{year}_S2.png')
        ax[int(year)-2016,0].imshow(img)
        ax[int(year)-2016,0].axis('off')


        img = plt.imread(f'{intermediate_dir}{year}_LULC.png')
        ax[int(year)-2016,1].imshow(img)
        ax[int(year)-2016,1].axis('off')

    chipid_str = data_dir.split('/')[-2]
    
    plt.suptitle(f"Land Use and Land Cover Maps of {chipid_str} from {min(years)} to {max(years)}", fontweight='bold', fontsize=8, y=1.000005)


    
    plt.tight_layout()
    fig.subplots_adjust(wspace=0.2)
    plt.savefig(f'{data_dir}massive_subplot.png',bbox_inches='tight')
    plt.close(fig)


def create_intermediate_yearly_subplots(chipid,years,data_dir='../plots/chips/',intermediate_dir='./'):
    plt.style.use('default')
    for year in years:

        fig, ax = plt.subplots(1,2, figsize=(16, 6.25))
        plt.tight_layout()
        
        img = plt.imread(f'{intermediate_dir}{year}_S2.png')
        ax[0].imshow(img)
        ax[0].axis('off')


        img = plt.imread(f'{intermediate_dir}{year}_LULC.png')
        ax[1].imshow(img)
        ax[1].axis('off')

        chipid_str = data_dir.split('/')[-2]
        plt.suptitle(f"Land Use and Land Cover Map of {chipid_str} for {year}", fontweight='bold', fontsize=20)

        plt.tight_layout()
                
        plt.savefig(f'{intermediate_dir}{year}_combined.png',bbox_inches='tight')
        plt.close(fig)


def create_directories(chipid,data_dir='../plots/chips/',selected=False):

    if selected:

        data_dir = data_dir + f'selected/{selected}/'
        # if the directory does not exist, create it
        if not os.path.exists(data_dir):
            os.mkdir(data_dir)

    chipid_str = '|'.join(sorted(chipid))
    if not os.path.exists(data_dir + f'{chipid_str}'):
        data_dir = data_dir + f'{chipid_str}/'
        os.mkdir(data_dir)

        gee_dir = data_dir+"support_files/"
        os.mkdir(gee_dir)
        
        intermediate_dir = data_dir + 'intermediate_plots/'
        os.mkdir(intermediate_dir)
    else:
        data_dir = data_dir + f'{chipid_str}/'
        gee_dir = data_dir+"support_files/"
        intermediate_dir = data_dir + 'intermediate_plots/'

    return data_dir,gee_dir,intermediate_dir,chipid_str

def create_gif_from_images(data_dir,years, duration=1000):

    images = ["intermediate_plots/"+f"{year}_combined.png" for year in years]
    images.sort()  # Ensure images are in order

    # Read the images
    frames = [Image.open(os.path.join(data_dir, image)) for image in images]

    # Save as a GIF
    if frames:
        frames[0].save(
            data_dir+"LULC.gif",
            format='GIF',
            append_images=frames[1:],
            save_all=True,
            duration=duration,
            loop=0
        )
    else:
        print("No PNG images found in the specified folder.")

def plot_chip_sankey(chipid,year_from,year_to,data_dir,country = "Denmark"):
    DB = DBMS()

    results = DB.read('GET_SINGLE_CHIP_GRAPH',{'_CHIPID_':chipid,
                                '_YEAR_FROM_':year_from,
                                '_YEAR_TO_':year_to,
                                '_AREA_':country})   

    if country == 'Israel':
        fac = 0.85
    else:
        fac = 1
    changed_area = results['changed_area'].sum()*fac
    
    plot_sankey(results,title=f"{chipid} - {round(changed_area,2)}km² changed ",outpath=f"{data_dir}{chipid}_sankey.png")



def plot_chip(chipid,years,country = "Denmark",selected = False):

    if not isinstance(chipid,list):
        chipid = [chipid]


    data_dir,gee_dir,intermediate_dir,chipid_str = create_directories(chipid,selected=selected)


    create_intermediate_plots(chipid,years,country=country,intermediate_dir=intermediate_dir,gee_dir=gee_dir)

    create_massive_subplot(chipid,years,data_dir,intermediate_dir)

    create_intermediate_yearly_subplots(chipid,years,data_dir,intermediate_dir)

    create_gif_from_images(data_dir, years)

    plot_chip_sankey(chipid[0],years[0],years[-1],data_dir,country=country)
    

In [ ]:
relevant_chips = {
                  "DEFORESTATION":[{"chipid":"14_3_12",
                                     "years":years,
                                     "country":"Denmark"}],
                  "RENEWABLE_EXPANSION":[{"chipid":"6_23_24",
                                     "years":years,
                                     "country":"Netherlands"}]
                 }


for change_type,chips in relevant_chips.items():
    for params in chips:
        params["selected"] = change_type
        plot_chip(**params)


In [ ]:
# OUTLIERS
outliers = pd.read_csv('../data/change_queries/outliers.csv')

cluster_country_mapping = {'0':'Israel','1':"Netherlands","2":"Estonia","3":"Denmark"}

for k,outlier_type in outliers[['country','cluster']].drop_duplicates().sort_values('cluster').iterrows():

    chosen_outlier = outliers[(outliers['country']==outlier_type['country']) & (outliers['cluster']==outlier_type['cluster'])].head(1)
    country = chosen_outlier['country'].values[0]
    chipid = chosen_outlier['chipid'].values[0]
    cluster = chosen_outlier['cluster'].values[0]
    print(f"Country : {country} - Chipid : {chipid} - outlier in cluster {cluster}, corresponding to {cluster_country_mapping[str(cluster)]}")
    
    plot_chip(chipid=chipid,
        years=years,
        country=country,
        selected="OUTLIERS")

In [ ]:
relevant_chips = {"DEFORESTATION":[{"chipid":"14_3_12",
                                     "years":years,
                                     "country":"Denmark"}],
                  "RENEWABLE_EXPANSION":[{"chipid":"6_23_24",
                                     "years":years,
                                     "country":"Netherlands"}]
                 }


for change_type,chips in relevant_chips.items():
    for params in chips:
        params["selected"] = change_type
        plot_chip(**params)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def PREZ_create_intermediate_yearly_subplots(chipid, years, data_dir='../plots/chips/', intermediate_dir='./'):
    plt.style.use('default')
    for year in years:
        fig, ax = plt.subplots(1, 2, figsize=(16, 6.25))
        
        img = plt.imread(f'{intermediate_dir}{year}_S2.png')
        ax[0].imshow(img)
        ax[0].axis('off')

        img = plt.imread(f'{intermediate_dir}{year}_LULC.png')
        ax[1].imshow(img)
        ax[1].axis('off')

        chipid_str = data_dir.split('/')[-2]
        plt.suptitle(f"Land Use and Land Cover Map -  chip {chipid} from 2016 - 2023", fontweight='bold', fontsize=25)

        # Create legend patches
        patches = [mpatches.Patch(color=color, label=label) for label, color in LEGEND_TO_PALETTE.items()]

        # Add legend below the plots with a maximum of 4 categories per row
        legend = fig.legend(handles=patches, loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.1))

        # Debugging: print legend to verify it's being created
        print(f"Legend created: {legend}")

        plt.tight_layout()  # Adjust rect to make space for the legend
        plt.subplots_adjust(bottom=0.1)  # Adjust bottom to make space for the legend

        plt.savefig(f'{intermediate_dir}{year}_combined.png', bbox_inches='tight')
        #plt.show()  # Display the plot to verify the legend
        #plt.close(fig)





In [ ]:
dirs = [i for i in os.listdir("../plots/chips/selected/") if i  != '.DS_Store']
dirs

In [ ]:



plt.style.use('default')
country_chip_map = {'1_3_16':{'country':'Israel',
                              'type':'solar'},
                    '1_3_19':{'country':'Israel',
                              'type':'desertification'},
                    '14_3_12':{'country':'Denmark',
                              'type':'deforestation'},
                    '6_23_24':{'country':'Netherlands',
                              'type':'solar_wind'},
                    '1_8_37':{'country':'Israel',
                              'type':'urbanization'},}




dirs = [i for i in os.listdir("../plots/chips/selected/") if i  != '.DS_Store']
for change_type in tqdm(dirs[:-1]):
    base_dir = f'../plots/chips/selected/{change_type}/'
    chipid  = [i for i in os.listdir(base_dir) if i  != '.DS_Store'][0]

    data_dir=f"{base_dir}/{chipid}/"

    intermediate_dir=data_dir+'intermediate_plots/'

    r = PREZ_create_intermediate_yearly_subplots(chipid, years,data_dir=data_dir, intermediate_dir=intermediate_dir)
    
    create_gif_from_images(data_dir, years)

    create_massive_subplot(chipid,years,data_dir,intermediate_dir)

    plot_chip_sankey(chipid,years[0],years[-1],data_dir,country=country_chip_map[chipid]['country'])

In [ ]:
[dirs[2],dirs[3]]

In [ ]:
data_dir='../plots/chips/selected/DESERTIFICATION/1_3_19/'
create_gif_from_images(data_dir, years)

In [ ]:
# Urbanization : 
### ISRAEL : "1_8_37"
### ISRAEL : "1_9_38"
### ISRAEL : "1_6_30"
### NETHERLANDS ; "6_12_17"

# Green Transition
### DENMARK : ""14_2_17""
### Netherlands : "6_13_17"
### Netherlands : "6_13_18"
### Netherlands : "6_14_18"
### Israel : "1_3_16"


# Desertification
### ISRAEL : "1_3_19" 
### ISRAEL : "1_2_19"

# Deforestation
### Denmark : ""14_3_12""
### Denmark : "14_2_16"
### ISRAEL : "1_7_25"
### ESTONIA : "6_11_18"

In [ ]:
years = [str(year) for year in range(2016,2024)]
chipid = "6_9_18"
country = "Netherlands"

plot_chip(chipid=chipid,
          years=years,
          country=country)

In [ ]:
# ESTONIA 
years = [str(year) for year in range(2016,2024)]
chipids = ["6_10_19","6_11_19"]
country = "Estonia"
for chipid in chipids:
    plot_chip(chipid=chipid,
            years=years,
            country=country)

In [ ]:
import pandas as pd
outliers = pd.read_csv('../data/change_queries/outliers.csv')


Country : Denmark - Chipid : 12_1_4 - outlier in cluster 2, corresponding to Estonia
Country : Denmark - Chipid : 13_4_7 - outlier in cluster 0, corresponding to Israel
Country : Israel - Chipid : 1_8_35 - outlier in cluster 1, corresponding to Netherlands
Country : Israel - Chipid : 1_12_40 - outlier in cluster 2, corresponding to Estonia
Country : Netherlands - Chipid : 6_13_11 - outlier in cluster 2, corresponding to Estonia


In [ ]:
# OUTLIERS
outliers = pd.read_csv('../data/change_queries/outliers.csv')

cluster_country_mapping = {'0':'Israel','1':"Netherlands","2":"Estonia","3":"Denmark"}
years = [str(year) for year in range(2016,2024)]

for k,outlier_type in outliers[['country','cluster']].drop_duplicates().sort_values('cluster').iterrows():
    chosen_outlier = outliers[(outliers['country']==outlier_type['country']) & (outliers['cluster']==outlier_type['cluster'])].head(1)
    country = chosen_outlier['country'].values[0]
    chipid = chosen_outlier['chipid'].values[0]
    cluster = chosen_outlier['cluster'].values[0]
    print(f"Country : {country} - Chipid : {chipid} - outlier in cluster {cluster}, corresponding to {cluster_country_mapping[str(cluster)]}")
    
    plot_chip(chipid=chipid,
        years=years,
        country=country)